<a href="https://colab.research.google.com/github/davidraamirez/GradientWithoutBackpropagation/blob/main/NN_fwd_gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split

In [2]:
%pip install torchmetrics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 3.2 MB/s eta 0:00:00


In [3]:
import torchmetrics
import torchvision

Loading and preprocessing the data

In [4]:
#Load the dataset
penguins = tfds.load('penguins', as_supervised=True, split='train')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/334 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/penguins/processed/1.0.0.incompleteZCFB6D/penguins-train.tfrecord*...:   0…

Dataset penguins downloaded and prepared to /root/tensorflow_datasets/penguins/processed/1.0.0. Subsequent calls will reuse this data.


In [5]:
# By default, the Dataset object is an iterator over the elements.
# The instructions below extract the underlying tensors.
X, y = penguins.batch(500).get_single_element()
X, y = X.numpy(), y.numpy()

In [6]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, stratify=y)

In [7]:
Xtrain = torch.from_numpy(Xtrain).float()
Xtest = torch.from_numpy(Xtest).float()

In [8]:
ytrain = torch.from_numpy(ytrain).long()
ytest = torch.from_numpy(ytest).long()

Define MultiLayer Neural Network

In [9]:
from torch import nn
from torch.nn import functional as F

In [106]:
class MNN(nn.Module):
  def __init__(self, input_size, fc1w, fc1b, fc2w, fc2b, fc3w, fc3b):
    super().__init__()

    self.w1 = nn.Parameter(fc1w)
    self.b1 = nn.Parameter(fc1b)
    
    self.w2 = nn.Parameter(fc2w)
    self.b2 = nn.Parameter(fc2b)

    self.w3 = nn.Parameter(fc3w)
    self.b3 = nn.Parameter(fc3b)

  def forward(self, x):
    x = F.relu(x@self.w1 + self.b1)
    x = F.relu(x@self.w2 + self.b2)
    x = x@self.w3 + self.b3
    x = x.reshape(1, -1)
    return torch.softmax(x, 1)

In [107]:
# We check if CUDA is available. If you do not see it,
# activate a GPU from Runtime >> Change runtime type and 
# restart the notebook.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


Initialize the parameters

In [108]:
# We initialize the parameters randomly and the model with an input size
fc1w = torch.FloatTensor(4, 64).uniform_(-1, 1)
fc1b = torch.FloatTensor(64).uniform_(-1, 1)
fc2w = torch.FloatTensor(64, 8).uniform_(-1, 1)
fc2b = torch.FloatTensor(8).uniform_(-1, 1)
fc3w = torch.FloatTensor(8, 3).uniform_(-1, 1)
fc3b = torch.FloatTensor(3).uniform_(-1, 1)
mnn = MNN(4, fc1w, fc1b, fc2w, fc2b, fc3w, fc3b).to(device)

In [109]:
# Note: we also need to move data when asking for a prediction
print(mnn(Xtrain[0].to(device)))

tensor([[0.1484, 0.7933, 0.0583]], grad_fn=<SoftmaxBackward0>)


Train and evaluate the network with forward gradient

In [110]:
def accuracy(ytrue, ypred):
  return (ypred.argmax(1) == ytrue).float().mean()

In [111]:
# Average accuracy at initialization is 10% (random guessing).
accuracy(ytrain.to(device), mnn(Xtrain.to(device)))

tensor(0.)

Define Cross-Entropy

In [112]:
def cross_entropyWfc1(fc1w, fc1b, fc2w, fc2b, fc3w, fc3b, ytrue, x):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],3))
  for j in range (ytrue.shape[0]):
    xj = x[j]
    xj = F.relu(xj@fc1w + fc1b)
    xj = F.relu(xj@fc2w + fc2b)
    xj = xj@fc3w + fc3b
    xj = xj.reshape(1, -1)
    ypred[j]=torch.softmax(xj, 1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue].log().mean()

In [114]:
def cross_entropyBfc1(fc1b, fc1w, fc2w, fc2b, fc3w, fc3b, ytrue, x):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],3))
  for j in range (ytrue.shape[0]):
    xj = x[j]
    xj = F.relu(xj@fc1w + fc1b)
    xj = F.relu(xj@fc2w + fc2b)
    xj = xj@fc3w + fc3b
    xj = xj.reshape(1, -1)
    ypred[j]=torch.softmax(xj, 1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue].log().mean()

In [115]:
def cross_entropyWfc2(fc2w, fc1b, fc1w, fc2b, fc3w, fc3b, ytrue, x):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],3))
  for j in range (ytrue.shape[0]):
    xj = x[j]
    xj = F.relu(xj@fc1w + fc1b)
    xj = F.relu(xj@fc2w + fc2b)
    xj = xj@fc3w + fc3b
    xj = xj.reshape(1, -1)
    ypred[j]=torch.softmax(xj, 1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue].log().mean()

In [116]:
def cross_entropyBfc2(fc2b, fc1b, fc2w, fc1w, fc3w, fc3b, ytrue, x):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],3))
  for j in range (ytrue.shape[0]):
    xj = x[j]
    xj = F.relu(xj@fc1w + fc1b)
    xj = F.relu(xj@fc2w + fc2b)
    xj = xj@fc3w + fc3b
    xj = xj.reshape(1, -1)
    ypred[j]=torch.softmax(xj, 1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue].log().mean()

In [117]:
def cross_entropyWfc3(fc3w, fc1b, fc2w, fc2b, fc1w, fc3b, ytrue, x):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],3))
  for j in range (ytrue.shape[0]):
    xj = x[j]
    xj = F.relu(xj@fc1w + fc1b)
    xj = F.relu(xj@fc2w + fc2b)
    xj = xj@fc3w + fc3b
    xj = xj.reshape(1, -1)
    ypred[j]=torch.softmax(xj, 1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue].log().mean()

In [118]:
def cross_entropyBfc3(fc3b, fc1b, fc2w, fc2b, fc3w, fc1w, ytrue, x):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],3))
  for j in range (ytrue.shape[0]):
    xj = x[j]
    xj = F.relu(xj@fc1w + fc1b)
    xj = F.relu(xj@fc2w + fc2b)
    xj = xj@fc3w + fc3b
    xj = xj.reshape(1, -1)
    ypred[j]=torch.softmax(xj, 1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue].log().mean()

In [119]:
print(cross_entropyWfc1(fc1w, fc1b, fc2w, fc2b, fc3w, fc3b, ytrain, Xtrain))
print(cross_entropyBfc1(fc1b, fc1w, fc2w, fc2b, fc3w, fc3b, ytrain, Xtrain))
print(cross_entropyWfc2(fc2w, fc1b, fc1w, fc2b, fc3w, fc3b, ytrain, Xtrain))
print(cross_entropyBfc2(fc2b, fc1b, fc2w, fc1w, fc3w, fc3b, ytrain, Xtrain))
print(cross_entropyWfc3(fc3w, fc1b, fc2w, fc2b, fc1w, fc3b, ytrain, Xtrain))
print(cross_entropyBfc3(fc3b, fc1b, fc2w, fc2b, fc3w, fc1w, ytrain, Xtrain))

tensor(1.4783)
tensor(1.4783)
tensor(1.4783)
tensor(1.4783)
tensor(1.4783)
tensor(1.4783)


In [120]:
def beale_function(x):
  return (torch.pow(torch.tensor([1.5])-x[0]+x[0]*x[1],2) + torch.pow(torch.tensor([2.25])-x[0]+x[0]*torch.pow(x[1],2),2)+torch.pow(torch.tensor([2.625])-x[0]+x[0]*torch.pow(x[1],3),2))

In [121]:
def rosenbrock_function(x):
  sum=0
  for p in x.size():
    for i in range (x.size(1)-1):
      sum += (100*torch.pow(x[i+1] - torch.pow(x[i], 2), 2) + torch.pow(x[i]-1, 2))
  return sum

In [122]:
from functorch import jvp
from functools import partial
import time 

In [136]:
def train_fwd_gradient(x, y):
  x, y = x.to(device), y.to(device)

  l_rate0 = 0.05

  #Parameters
  fc1w = torch.FloatTensor(4, 64).uniform_(-1, 1)
  fc1b = torch.FloatTensor(64).uniform_(-1, 1)
  fc2w = torch.FloatTensor(64, 8).uniform_(-1, 1)
  fc2b = torch.FloatTensor(8).uniform_(-1, 1)
  fc3w = torch.FloatTensor(8, 3).uniform_(-1, 1)
  fc3b = torch.FloatTensor(3).uniform_(-1, 1)

  loss = cross_entropyWfc1(fc1w, fc1b, fc2w, fc2b, fc3w, fc3b, ytrain, Xtrain)

  #mnn = MNN(4, fc1w, fc1b, fc2w, fc2b, fc3w, fc3b).to(device)

  t=0
  t0 = time.time()

  while (loss>0.2) :

    vfc1w=torch.randn(fc1w.shape)
    vfc1w=(vfc1w - torch.mean(vfc1w))/torch.std(vfc1w)
    vfc1b=torch.randn(fc1b.shape)
    vfc1b=(vfc1b - torch.mean(vfc1b))/torch.std(vfc1b)
    vfc2w=torch.randn(fc2w.shape)
    vfc2w=(vfc2w - torch.mean(vfc2w))/torch.std(vfc2w)
    vfc2b=torch.randn(fc2b.shape)
    vfc2b=(vfc2b - torch.mean(vfc2b))/torch.std(vfc2b)
    vfc3w=torch.randn(fc3w.shape)
    vfc3w=(vfc3w - torch.mean(vfc3w))/torch.std(vfc3w)
    vfc3b=torch.randn(fc3b.shape)
    vfc3b=(vfc3b - torch.mean(vfc3b))/torch.std(vfc3b)

    fw1 = partial(cross_entropyWfc1, fc1b=fc1b, fc2w=fc2w, fc2b=fc2b, fc3w=fc3w, fc3b=fc3b, ytrue=ytrain, x=Xtrain)
    fb1 = partial(cross_entropyBfc1, fc1w=fc1w, fc2w=fc2w, fc2b=fc2b, fc3w=fc3w, fc3b=fc3b, ytrue=ytrain, x=Xtrain)
    fw2 = partial(cross_entropyWfc2, fc1b=fc1b, fc1w=fc1w, fc2b=fc2b, fc3w=fc3w, fc3b=fc3b, ytrue=ytrain, x=Xtrain)
    fb2 = partial(cross_entropyBfc2, fc1b=fc1b, fc2w=fc2w, fc1w=fc1w, fc3w=fc3w, fc3b=fc3b, ytrue=ytrain, x=Xtrain)
    fw3 = partial(cross_entropyWfc3, fc1b=fc1b, fc2w=fc2w, fc2b=fc2b, fc1w=fc1w, fc3b=fc3b, ytrue=ytrain, x=Xtrain)
    fb3 = partial(cross_entropyBfc3, fc1b=fc1b, fc2w=fc2w, fc2b=fc2b, fc3w=fc3w, fc1w=fc1w, ytrue=ytrain, x=Xtrain)

    loss, dtw1=jvp(fw1,(fc1w, ), (vfc1w, ))
    ftb1, dtb1=jvp(fb1,(fc1b, ), (vfc1b, ))
    ftw2, dtw2=jvp(fw2,(fc2w, ), (vfc2w, ))
    ftb2, dtb2=jvp(fb2,(fc2b, ), (vfc2b, ))
    ftw3, dtw3=jvp(fw3,(fc3w, ), (vfc3w, ))
    ftb3, dtb3=jvp(fb3,(fc3b, ), (vfc3b, ))

    gtw1 = vfc1w*dtw1
    gtb1 = vfc1b*dtb1
    gtw2 = vfc2w*dtw2
    gtb2 = vfc2b*dtb2
    gtw3 = vfc3w*dtw3
    gtb3 = vfc3b*dtb3

    fc1w -= l_rate0*gtw1
    fc1b -= l_rate0*gtb1
    fc2w -= l_rate0*gtw2
    fc2b -= l_rate0*gtb2
    fc3w -= l_rate0*gtw3
    fc3b -= l_rate0*gtb3


    #mnn = MNN(4, fc1w, fc1b, fc2w, fc2b, fc3w, fc3b).to(device)
    t1=time.time()
    t+=t1-t0
    t0=t1
    print('Time', t, 'loss', loss)

  return fc1w, fc1b, fc2w, fc2b, fc3w, fc3b 

In [137]:
fc1w, fc1b, fc2w, fc2b, fc3w, fc3b = train_fwd_gradient(Xtrain, ytrain)

dtw1 tensor(0.2296)
Time 0.6264793872833252 loss tensor(1.2046)
dtw1 tensor(0.6983)
Time 1.270463466644287 loss tensor(1.0241)
dtw1 tensor(0.4844)
Time 1.891709566116333 loss tensor(0.9423)
dtw1 tensor(0.8665)
Time 2.5325214862823486 loss tensor(0.8115)
dtw1 tensor(0.3615)
Time 3.141702890396118 loss tensor(0.7631)
dtw1 tensor(-0.1554)
Time 3.756065607070923 loss tensor(0.7433)
dtw1 tensor(0.2694)
Time 4.398963451385498 loss tensor(0.7602)
dtw1 tensor(-0.4476)
Time 5.000510215759277 loss tensor(0.7285)
dtw1 tensor(-0.2278)
Time 5.629626750946045 loss tensor(0.6812)
dtw1 tensor(0.1373)
Time 6.228475332260132 loss tensor(0.6678)
dtw1 tensor(-0.2884)
Time 6.843095779418945 loss tensor(0.6465)
dtw1 tensor(0.5434)
Time 7.770217418670654 loss tensor(0.6247)
dtw1 tensor(0.2266)
Time 8.72756838798523 loss tensor(0.5847)
dtw1 tensor(-0.1054)
Time 9.714744329452515 loss tensor(0.5820)
dtw1 tensor(0.4624)
Time 10.324612140655518 loss tensor(0.5814)
dtw1 tensor(-0.0299)
Time 10.922689199447632 los

In [146]:
mnn = MNN(4, fc1w, fc1b, fc2w, fc2b, fc3w, fc3b)
ypred=torch.randn(Xtest.size(0),3)
error=0
for i in range (Xtest.size(0)):
  ypred[i]=mnn(Xtest[i])
  if (mnn(Xtest[i]).argmax(1)- ytest[i])!=0:
    error = error+ 1
error= error / Xtest.size(0)

In [147]:
print(error)

0.023809523809523808


In [148]:
print(accuracy(ytest, ypred))

tensor(0.9762)
